<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Libraries</a></span></li></ul></li><li><span><a href="#Import-Data" data-toc-modified-id="Import-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Data</a></span></li></ul></div>

Use a hidden Markov model to create a part of speech tagger using the [pomegranate](https://pomegranate.readthedocs.io/en/latest/) library

## Libraries

In [3]:
# Jupyter "magic methods"
%load_ext autoreload
%aimport helpers, tests
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import os

import matplotlib.pyplot as plt
import numpy as np

from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict
from helpers import show_model, Dataset
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

# Import Data

Load in text from the [Brown University Standard Corpus of Present-Day American English](https://en.wikipedia.org/wiki/Brown_Corpus)

In [9]:
# data paths
brown_data = os.path.join("data","brown-universal.txt")
tags_data = os.path.join("data","tags-universal.txt")

In [13]:
# check out the brown data
with open(brown_data, "r") as f:
    text = f.read()
    
print(text[:150])

b100-5507
Mr.	NOUN
Podger	NOUN
had	VERB
thanked	VERB
him	PRON
gravely	ADV
,	.
and	CONJ
now	ADV
he	PRON
made	VERB
use	NOUN
of	ADP
the	DET
advice	NOUN
.


In [14]:
# check out the universal tags list
with open(tags_data, "r") as f:
    tags = f.read()
    
print(tags)

.
ADJ
ADP
ADV
CONJ
DET
NOUN
NUM
PRON
PRT
VERB
X
